### LangGraph

#### Config the model

In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI

model= ChatGoogleGenerativeAI(model="gemini-1.5-flash")
output= model.invoke("Hi")
print(output.content)

Hi there! How can I help you today?


#### Config the Embedding Model

In [39]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings= HuggingFaceEmbeddings(model_name="BAAI/bge-small-en")

In [40]:
embeddings

HuggingFaceEmbeddings(model_name='BAAI/bge-small-en', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False)

In [41]:
embeddings.embed_query("Hi")

[-0.07908844202756882,
 -0.04442884773015976,
 0.05409764125943184,
 -0.044667042791843414,
 -0.004612544551491737,
 0.02221708931028843,
 0.059072185307741165,
 0.04883622005581856,
 0.028120674192905426,
 0.0176496971398592,
 0.0028322997968643904,
 -0.08215256780385971,
 0.034488774836063385,
 0.015403807163238525,
 0.028747456148266792,
 -0.007119529414921999,
 0.029838014394044876,
 -0.018075022846460342,
 -0.09786265343427658,
 0.021732240915298462,
 0.019262226298451424,
 -0.01266544871032238,
 -0.043377310037612915,
 -0.032973047345876694,
 0.008817197754979134,
 0.008439965546131134,
 -0.041041988879442215,
 -0.01432439312338829,
 -0.024374419823288918,
 -0.07668082416057587,
 -0.035451490432024,
 0.005375921260565519,
 0.04354889690876007,
 0.006629592273384333,
 0.03401279076933861,
 -0.025910746306180954,
 0.0015285558765754104,
 0.00927803386002779,
 -0.04985249787569046,
 0.01580633781850338,
 0.04987366124987602,
 -0.029326140880584717,
 -0.02101047709584236,
 -0.0477381

In [42]:
len(embeddings.embed_query("Hi"))

384

#### Lets take a data embed it and store in VDB (Vector Database)

In [37]:
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [17]:
loader= DirectoryLoader("../data2", glob="./*.txt", loader_cls=TextLoader)

In [20]:
docs= loader.load()

In [21]:
docs

[Document(metadata={'source': '..\\data2\\usa.txt'}, page_content="🇺🇸 Overview of the U.S. Economy\nThe United States of America possesses the largest economy in the world in terms of nominal GDP, making it the most powerful economic force globally. It operates under a capitalist mixed economy, where the private sector dominates, but the government plays a significant regulatory and fiscal role. With a population of over 335 million people and a high level of technological advancement, the U.S. economy thrives on a foundation of consumer spending, innovation, global trade, and financial services. It has a highly diversified structure with strong sectors in technology, healthcare, finance, real estate, defense, and agriculture.\n\nU.S. GDP – Size, Composition, and Global Share\nAs of 2024, the United States’ nominal GDP is estimated to be around $28 trillion USD, accounting for approximately 25% of the global economy. It ranks #1 in the world by nominal GDP, far ahead of China (which ra

In [23]:
len(docs)

1

In [22]:
docs[0].page_content

"🇺🇸 Overview of the U.S. Economy\nThe United States of America possesses the largest economy in the world in terms of nominal GDP, making it the most powerful economic force globally. It operates under a capitalist mixed economy, where the private sector dominates, but the government plays a significant regulatory and fiscal role. With a population of over 335 million people and a high level of technological advancement, the U.S. economy thrives on a foundation of consumer spending, innovation, global trade, and financial services. It has a highly diversified structure with strong sectors in technology, healthcare, finance, real estate, defense, and agriculture.\n\nU.S. GDP – Size, Composition, and Global Share\nAs of 2024, the United States’ nominal GDP is estimated to be around $28 trillion USD, accounting for approximately 25% of the global economy. It ranks #1 in the world by nominal GDP, far ahead of China (which ranks 2nd). The U.S. GDP per capita is also among the highest, hover

In [24]:
text_splitter= RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=50
)

In [25]:
text_splitter

In [27]:
new_docs= text_splitter.split_documents(documents= docs)

In [28]:
new_docs

[Document(metadata={'source': '..\\data2\\usa.txt'}, page_content='🇺🇸 Overview of the U.S. Economy'),
 Document(metadata={'source': '..\\data2\\usa.txt'}, page_content='The United States of America possesses the largest economy in the world in terms of nominal GDP, making it the most powerful economic force globally. It operates under a capitalist mixed economy,'),
 Document(metadata={'source': '..\\data2\\usa.txt'}, page_content='It operates under a capitalist mixed economy, where the private sector dominates, but the government plays a significant regulatory and fiscal role. With a population of over 335 million people and a'),
 Document(metadata={'source': '..\\data2\\usa.txt'}, page_content='a population of over 335 million people and a high level of technological advancement, the U.S. economy thrives on a foundation of consumer spending, innovation, global trade, and financial services.'),
 Document(metadata={'source': '..\\data2\\usa.txt'}, page_content='innovation, global trade,

In [29]:
len(new_docs)

55

In [30]:
new_docs[0].page_content

'🇺🇸 Overview of the U.S. Economy'

In [33]:
doc_string= [doc.page_content for doc in new_docs]

In [34]:
doc_string

['🇺🇸 Overview of the U.S. Economy',
 'The United States of America possesses the largest economy in the world in terms of nominal GDP, making it the most powerful economic force globally. It operates under a capitalist mixed economy,',
 'It operates under a capitalist mixed economy, where the private sector dominates, but the government plays a significant regulatory and fiscal role. With a population of over 335 million people and a',
 'a population of over 335 million people and a high level of technological advancement, the U.S. economy thrives on a foundation of consumer spending, innovation, global trade, and financial services.',
 'innovation, global trade, and financial services. It has a highly diversified structure with strong sectors in technology, healthcare, finance, real estate, defense, and agriculture.',
 'U.S. GDP – Size, Composition, and Global Share',
 'As of 2024, the United States’ nominal GDP is estimated to be around $28 trillion USD, accounting for approximately 

In [35]:
len(doc_string)

55

In [44]:
db= Chroma.from_documents(new_docs, embeddings)

In [47]:
db.as_retriever(search_kwargs={"k": 3})

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000002BDD1335090>, search_kwargs={'k': 3})

In [49]:
retriever= db.as_retriever(search_kwargs={"k": 3})

In [50]:
retriever.invoke("industrial growth of usa?")

[Document(metadata={'source': '..\\data2\\usa.txt'}, page_content='Looking forward, the U.S. economy is expected to grow at a moderate pace, powered by innovation in AI, green energy, robotics, biotech, and quantum computing. The Biden administration’s Inflation'),
 Document(metadata={'source': '..\\data2\\usa.txt'}, page_content='🇺🇸 Overview of the U.S. Economy'),
 Document(metadata={'source': '..\\data2\\usa.txt'}, page_content='The U.S. economy remains the engine of global growth, backed by unmatched innovation, financial dominance, and a strong institutional framework. Its $28 trillion GDP and influence over global')]